In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -q -y psi4 python=3.7 -c psi4

# install rdkit
!conda install -q -y rdkit python=3.7 -c rdkit

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

# this command is needed to avoid "Loader" error.
!pip install distributed

--2022-04-25 08:20:38--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-25 08:20:38--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M  84.2MB/s    in 0.9s    

2022-04-25 08:20:39 (84.2 MB/s) - ‘Miniconda3-latest-Linu

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole
# check rdkit version
print('rdkit version: ', rdBase.rdkitVersion)

rdkit version:  2020.09.1


In [4]:
## convert SMILES to xyz format
def smi2xyz_neutral(smiles):
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles)) # make mol object and add Hydrogen
    AllChem.EmbedMolecule(mol, AllChem.ETKDGv3()) # expand to 3D
    AllChem.MMFFOptimizeMolecule(mol) # structure optimization by MMFF
    #xyz = Chem.MolToXYZBlock(mol)
    conf = mol.GetConformer(-1)

    plus = smiles.count('+')
    minus = smiles.count('-')
    charge = plus - minus + 0 # total charge
    
    multi = (charge%2) + 1 # 2S+1 = 2*((1/2)*(charge%2)) + 1
    
    xyz = str(charge) + " " + str(multi)
    for atom, (x,y,z) in zip(mol.GetAtoms(), conf.GetPositions()):
        xyz += '\n'
        xyz += '{} {} {} {}'.format(atom.GetSymbol(), x, y, z)
        #xyz += '{}\t{}\t{}\t{}'.format(atom.GetSymbol(), x, y, z)
        
    return xyz

In [5]:
smiles = 'COCCc1ccccc1'
#smiles = 'N' # test NH3

In [6]:
xyz_neutral = smi2xyz_neutral(smiles)
mol_neutral = psi4.geometry(xyz_neutral)
print(mol_neutral.save_string_xyz())

0 1
 C    3.371581556826    0.639688607986    1.347370696156
 O    2.329677185794   -0.240249594507    0.947940100669
 C    2.083778057010   -0.161051071365   -0.459088401900
 C    0.910584598968   -1.068971823930   -0.833771685188
 C   -0.432301744791   -0.450257930510   -0.516914069724
 C   -0.897009936033   -0.399972161343    0.804205555849
 C   -2.132859712240    0.176847561505    1.100125971368
 C   -2.916349136641    0.709965140289    0.078689201286
 C   -2.463717632362    0.667954177441   -1.238719329795
 C   -1.227155901756    0.091867760222   -1.536204397166
 H    3.096152061360    1.678135554093    1.139732106519
 H    4.304707909070    0.386931527620    0.834798023308
 H    3.522823744181    0.527708466378    2.424306223140
 H    1.887511470368    0.877886841232   -0.750110183317
 H    2.983048577138   -0.509142963631   -0.980529007503
 H    0.986632891744   -2.031987348041   -0.313796951786
 H    0.956707767641   -1.284918794160   -1.908246015898
 H   -0.290398512574   -0.8

In [7]:
## convert SMILES to xyz format
def smi2xyz_oxidized(smiles):
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles)) # make mol object and add Hydrogen
    AllChem.EmbedMolecule(mol, AllChem.ETKDGv3()) # expand to 3D
    AllChem.MMFFOptimizeMolecule(mol) # structure optimization by MMFF
    #xyz = Chem.MolToXYZBlock(mol)
    conf = mol.GetConformer(-1)

    plus = smiles.count('+')
    minus = smiles.count('-')
    charge = plus - minus + 1 # total charge
    
    multi = (charge%2) + 1 # 2S+1 = 2*((1/2)*(charge%2)) + 1
    
    xyz = str(charge) + " " + str(multi)
    for atom, (x,y,z) in zip(mol.GetAtoms(), conf.GetPositions()):
        xyz += '\n'
        xyz += '{} {} {} {}'.format(atom.GetSymbol(), x, y, z)
        #xyz += '{}\t{}\t{}\t{}'.format(atom.GetSymbol(), x, y, z)
        
    return xyz

In [8]:
xyz_oxidized = smi2xyz_oxidized(smiles)
mol_oxidized = psi4.geometry(xyz_oxidized)
print(mol_oxidized.save_string_xyz())

1 2
 C   -3.827397720679   -0.820796295871   -0.326384837409
 O   -3.203891233917    0.379671067992    0.109260656119
 C   -1.833000636912    0.475237025139   -0.284920436885
 C   -0.921562372552   -0.316025975550    0.654483125310
 C    0.535569815651   -0.129524468239    0.317452851900
 C    1.269895469786    0.913483699534    0.897366374107
 C    2.613932847568    1.098639782831    0.570146852759
 C    3.234447054470    0.246521059296   -0.341393113275
 C    2.511540894763   -0.790783403262   -0.927575359315
 C    1.167243661581   -0.977717983583   -0.601913424795
 H   -3.782210590904   -0.904359833842   -1.416465540562
 H   -4.878135344138   -0.788106355054   -0.025449645244
 H   -3.364629695854   -1.694958783582    0.138899019434
 H   -1.712566430280    0.162759061357   -1.329593840431
 H   -1.569836576384    1.538408417806   -0.240687407611
 H   -1.169058965418   -1.383816937500    0.623917527245
 H   -1.108886811270   -0.015429484564    1.693459737201
 H    0.798307266489    1.5

In [9]:
psi4.core.set_output_file("log_neutral.txt")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

# input (PCM Solver)
pcm = '''
units = angstrom
medium
{
    solvertype = iefpcm
    solvent = acetonitrile
}
cavity
{
    type = gepol
    scaling = True
    radiiset = uff
    mode = implicit
}
'''
# acetonitrile solvet (epsilon = 37.5)

psi4.set_options({'pcm': True})

#psi4.pcm_helper(pcm)
#RHF, ROHF, UHF, CUHF, RKS, UKS (Default: RHF)
#psi4.set_options({'pcm': True, 'reference': 'UHF'}) # HF, MP2 
#level = 'HF/STO-3G' # RHF, ROHF, UHF, CUHF
#level = 'MP2/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'MP2.5/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'MP3/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'CCSD(T)/cc-pVTZ' # RHF, ROHF, UHF, CUHF

psi4.set_options({'pcm': True, 'reference': 'UKS'}) # DFT
level = 'B3LYP/6-31+G(d,p)' # RKS, UKS

psi4.pcm_helper(pcm)

scf_neutral_e, neutral_wfn = psi4.energy(level, return_wfn = True, molecule=mol_neutral)
#scf_neutral_e, neutral_wfn = psi4.optimize(level, return_wfn = True, molecule=mol_neutral)
#scf_neutral_e, neutral_wfn = psi4.frequency(level, return_wfn = True, molecule=mol_neutral, ref_gradient=neutral_wfn.gradient())
print('energy at the {} level of theory:\t{:.4f} [Ha]'.format(level, scf_neutral_e))

energy at the B3LYP/6-31+G(d,p) level of theory:	-425.4302 [Ha]


In [10]:
!cat log_neutral.txt


  Memory set to   9.313 GiB by Python driver.
  Threads set to 2 by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/
  PCM does not make use of molecular symmetry: further calculations in C1 point group.

*** tstart() called on 6adaf043c345
*** at Mon Apr 25 08:24:50 2022

   => Loading Basis Set <=

    Name: 6-31+G(D,P)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1, 3-10 entry C          line   123 file /usr/local/share/psi4/basis/6-31pg_d_p_.gbs 
    atoms 2       entry O          line   161 file /usr/local/share/psi4/basis/6-31pg_d_p_.gbs 
    atoms 11-22   entry H          line    46 file /usr/local/share/psi4/basis/6-31pg_d_p_.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UKS Reference
                        2 Threads,   9536 MiB Core
         ----------

In [11]:
#ld_neutral = open('log_neutral.txt')
#lines = ld_neutral.readlines()
#for line in lines:
#    if line.find("Total") >= 0 and line.find("H,") >= 0:
#        H_neutral = line.split()[6]
#    if line.find("Total") >= 0 and line.find("G,") >= 0:
#        G_neutral = line.split()[7]
#print("Total H [Ha], Enthalpy at  298.15 [K] =",H_neutral)
#print("Total G [Ha], Free enthalpy at  298.15 [K] =",G_neutral)

In [12]:
psi4.core.set_output_file("log_oxidized.txt")
scf_oxidized_e, oxidized_wfn = psi4.energy(level, return_wfn = True, molecule=mol_oxidized)
#scf_oxidized_e, oxidized_wfn = psi4.optimize(level, return_wfn = True, molecule=mol_oxidized)
#scf_oxidized_e, oxidized_wfn = psi4.frequency(level, return_wfn = True, molecule=mol_oxidized, ref_gradient=oxidized_wfn.gradient())
print('energy at the {} level of theory:\t{:.4f} [Ha]'.format(level, scf_oxidized_e))

energy at the B3LYP/6-31+G(d,p) level of theory:	-425.1808 [Ha]


In [13]:
!cat log_oxidized.txt


Scratch directory: /tmp/

Scratch directory: /tmp/
  PCM does not make use of molecular symmetry: further calculations in C1 point group.

*** tstart() called on 6adaf043c345
*** at Mon Apr 25 08:46:39 2022

   => Loading Basis Set <=

    Name: 6-31+G(D,P)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1, 3-10 entry C          line   123 file /usr/local/share/psi4/basis/6-31pg_d_p_.gbs 
    atoms 2       entry O          line   161 file /usr/local/share/psi4/basis/6-31pg_d_p_.gbs 
    atoms 11-22   entry H          line    46 file /usr/local/share/psi4/basis/6-31pg_d_p_.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UKS Reference
                        2 Threads,   9536 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular p

In [14]:
#ld_oxidized = open('log_oxidized.txt')
#lines = ld_oxidized.readlines()
#for line in lines:
#    if line.find("Total") >= 0 and line.find("H,") >= 0:
#        H_oxidized = line.split()[6]
#    if line.find("Total") >= 0 and line.find("G,") >= 0:
#        G_oxidized = line.split()[7]
#print("Total H [Ha], Enthalpy at  298.15 [K] =",H_oxidized)
#print("Total G [Ha], Free enthalpy at  298.15 [K] =",G_oxidized)

In [15]:
NHE = 4.28 # [V]
# F = 96485.3321233100184 # [C/mol] = [(J/V)/mol]
# 1 [J/mol] = 1.03636E-05 [eV]
F = 96485.3321233100184*1.03636E-05 # [e]
# 1 [Ha] = 13.6058*2 [eV], Ha = Hartree
Eox = (scf_oxidized_e - scf_neutral_e)*13.6058*2/(1*F) - NHE # The right way is to use Gibbs free energy
#Eox = (G_oxidized - G_neutral)*13.6058*2/(1*F) - NHE
print('Eox = {:.4f} [V]'.format(Eox))

Eox = 2.5068 [V]
